In [1]:
import requests
import pandas as pd
import os
import time
from datetime import datetime

In [2]:
user = "CSSEGISandData"
repo = "COVID-19"

url = "https://api.github.com/repos/{}/{}/git/trees/master?recursive=1".format(user, repo)
urlRaw = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"

r = requests.get(url)
res = r.json()
i = 0

start_time = time.time()

for file in res["tree"]:
    pathToCheck = file["path"]
    if i == 10000:
        break
    if pathToCheck.endswith('.csv') \
            and pathToCheck.find('csse_covid_19_daily_reports')!=-1 \
            and pathToCheck.find('_us')==-1 \
            and not os.path.isfile("data/" + file["path"]):
            #r = requests.get(urlRaw + file["path"])
            r = requests.head(urlRaw + file["path"])
            if int(r.headers['Content-Length']) > 50000:
                r = requests.get(urlRaw + file["path"])
                open("data/" + file["path"], 'wb').write(r.content)
                i += 1
            if i%10 == 0 and i > 0:
                print(f'{i} files downloaded in {(time.time() - start_time)} seconds')
                start_time = time.time()

10 files downloaded in 7.7316975593566895 seconds
20 files downloaded in 10.300618410110474 seconds
30 files downloaded in 11.704235076904297 seconds
30 files downloaded in 0.29028820991516113 seconds
40 files downloaded in 13.294408321380615 seconds
40 files downloaded in 0.28428220748901367 seconds
50 files downloaded in 11.254976034164429 seconds
50 files downloaded in 0.28218841552734375 seconds
60 files downloaded in 12.30478572845459 seconds
60 files downloaded in 0.28188014030456543 seconds
70 files downloaded in 11.500237464904785 seconds
70 files downloaded in 0.3149597644805908 seconds
80 files downloaded in 10.77123737335205 seconds
90 files downloaded in 8.773683309555054 seconds
100 files downloaded in 9.379316568374634 seconds
